<a href="https://colab.research.google.com/github/toskpl/dw_matrix_car/blob/master/day4_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 3.5MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 3.4MB/s 


In [0]:
""" wczytanie bibliotek"""

import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb
 
from sklearn.metrics import mean_absolute_error as mea
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [4]:
cd '/content/drive/My Drive/Colab Notebooks/DW_Matrix/matrix_two/dw_matrix_car'


/content/drive/My Drive/Colab Notebooks/DW_Matrix/matrix_two/dw_matrix_car


In [5]:
!ls

data  day2_visualization.ipynb	day3_simple_model.ipynb  LICENSE  README.md


## Wczytanie danych

In [6]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

## Feature Engineering

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
  if isinstance (df[feat][0],list): continue

  factorize_values = df[feat].factorize()[0]
  """ zabezpieczenie aby mozna bylo uruchomic wiwel razy komorke"""
  if SUFFIX_CAT in feat:
     df[feat] = factorize_values
  else:
    df [feat + SUFFIX_CAT] = df[feat].factorize()[0]

In [9]:
"""budujemy zbior cech"""

cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [12]:
X = df [cat_feats].values

y= df ['price_value'].values

model = DecisionTreeRegressor(max_depth=5)

scores = cross_val_score(model, X,y, cv=3, scoring = 'neg_mean_absolute_error')
np.mean(scores), np.std(scores)

(-19695.13091100928, 148.72570644015937)

In [0]:
"""funkcja pomocnicza"""

def run_model(model, feats):
  X = df [feats].values
  y= df ['price_value'].values

  scores = cross_val_score(model, X,y, cv=3, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores) 

## DecisionTreeRegressor

In [14]:
run_model(DecisionTreeRegressor(max_depth=5),cat_feats)

(-19695.13091100928, 148.72570644015792)

## RandomForestRegressor


In [19]:
model = RandomForestRegressor(max_depth=5,n_estimators= 50, random_state=2020)
run_model(model,cat_feats)

(-18774.078311945963, 56.07799419652943)

## XGBoost


In [27]:
xgb_params = {
    'max_depth': 5,
    'n_estimators': 50,
    'learning_rate' : 0.1,
    'seed': 2020
}

model = xgb.XGBRegressor(**xgb_params)
run_model(model,cat_feats)

[04:52:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:52:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:52:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [29]:
"""ważnosć cech """
m = xgb.XGBRegressor(max_depth = 5,n_estimators = 50, learning_rate = 0.1, seed = 2020)
m.fit (X,y)

imp = PermutationImportance(m, random_state= 0).fit(X,y)
eli5.show_weights(imp,feature_names = cat_feats)

[05:03:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd__cat
0.1132 ± 0.0032,param_rok-produkcji__cat
0.1090 ± 0.0025,param_stan__cat
0.0619 ± 0.0025,param_skrzynia-biegów__cat
0.0568 ± 0.0016,param_faktura-vat__cat
0.0489 ± 0.0014,param_moc__cat
0.0273 ± 0.0007,param_marka-pojazdu__cat
0.0242 ± 0.0013,feature_kamera-cofania__cat
0.0212 ± 0.0008,param_typ__cat
0.0174 ± 0.0008,param_pojemność-skokowa__cat


In [0]:
feats = ['param_rok-produkcji__cat',
         'param_napęd__cat',
         'param_stan__cat',
         'param_skrzynia-biegów__cat',
         'param_faktura-vat__cat',
         'param_moc__cat',
         'param_marka-pojazdu__cat',
         'feature_kamera-cofania__cat',
         'param_typ__cat',
         'param_pojemność-skokowa__cat',
         'seller_name__cat',
         'feature_wspomaganie-kierownicy__cat',
         'param_model-pojazdu__cat',
         'param_wersja__cat',
         'param_kod-silnika__cat',
         'feature_system-start-stop__cat',
         'feature_asystent-pasa-ruchu__cat',
         'feature_czujniki-parkowania-przednie__cat',
         'feature_łopatki-zmiany-biegów__cat',
         'feature_regulowane-zawieszenie__cat']

In [36]:
len(feats)

20

In [32]:
""" model na 20 najistotniejszych cechach """
""" wynik (-13373.448626945092, 66.39290420220289)"""
model = xgb.XGBRegressor(**xgb_params)
run_model(model,feats)

[05:17:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:17:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:18:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13373.448626945092, 66.39290420220289)

In [40]:
##parametr param_rok-produkcji
df['param_rok-produkcji'].unique()

array(['2018', '2011', '2015', '2009', '2017', '2012', '2013', '2007',
       '2001', '2016', '2006', '2008', '2004', '1999', '2000', '2010',
       '2005', '2002', '1998', '2014', '2003', '1982', '1995', '1997',
       '1992', '1993', '1994', '1996', '1989', '1988', '1967', '1987',
       '1970', '1959', '1990', '1991', '1974', None, '1975', '1973',
       '1953', '1985', '1984', '1986', '1981', '1979', '1960', '1983',
       '1978', '1964', '1980', '1972', '1969', '1956', '1966', '1977',
       '1962', '1965', '1971', '1963', '1961', '1952', '1949', '1976',
       '1937', '1968', '1958', '1955', '1933', '1929', '1957', '1944',
       '1954', '1932', '1936', '1947', '1948'], dtype=object)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x : -1 if str(x) == 'None' else int(x) )

In [0]:
## cecha param_rok-produkcji po przekszałceniu
feats = ['param_rok-produkcji',
         'param_napęd__cat',
         'param_stan__cat',
         'param_skrzynia-biegów__cat',
         'param_faktura-vat__cat',
         'param_moc__cat',
         'param_marka-pojazdu__cat',
         'feature_kamera-cofania__cat',
         'param_typ__cat',
         'param_pojemność-skokowa__cat',
         'seller_name__cat',
         'feature_wspomaganie-kierownicy__cat',
         'param_model-pojazdu__cat',
         'param_wersja__cat',
         'param_kod-silnika__cat',
         'feature_system-start-stop__cat',
         'feature_asystent-pasa-ruchu__cat',
         'feature_czujniki-parkowania-przednie__cat',
         'feature_łopatki-zmiany-biegów__cat',
         'feature_regulowane-zawieszenie__cat']

In [49]:
""" wynik (-11308.577496510174, 28.11473643563055)"""
run_model(model,feats)

[07:32:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:32:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:33:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11308.577496510174, 28.11473643563055)

In [50]:
##parametr param_moc
df['param_moc'].unique()

array(['90 KM', '115 KM', '262 KM', '110 KM', '310 KM', '105 KM',
       '140 KM', '175 KM', '125 KM', '185 KM', '190 KM', '440 KM',
       '141 KM', '200 KM', '224 KM', '75 KM', '99 KM', '184 KM', '109 KM',
       '233 KM', '116 KM', '68 KM', '286 KM', '126 KM', '160 KM',
       '135 KM', '120 KM', '272 KM', None, '150 KM', '180 KM', '136 KM',
       '102 KM', '131 KM', '218 KM', '245 KM', '170 KM', '112 KM',
       '250 KM', '252 KM', '73 KM', '100 KM', '313 KM', '101 KM',
       '285 KM', '70 KM', '383 KM', '174 KM', '277 KM', '132 KM',
       '130 KM', '215 KM', '60 KM', '330 KM', '163 KM', '177 KM', '98 KM',
       '78 KM', '189 KM', '156 KM', '143 KM', '69 KM', '113 KM', '65 KM',
       '122 KM', '82 KM', '251 KM', '95 KM', '197 KM', '235 KM', '238 KM',
       '171 KM', '381 KM', '400 KM', '178 KM', '80 KM', '165 KM', '85 KM',
       '258 KM', '142 KM', '204 KM', '124 KM', '55 KM', '144 KM',
       '231 KM', '248 KM', '152 KM', '181 KM', '210 KM', '340 KM',
       '129 KM', '147 

In [63]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None'  else x)
df['param_moc'].unique

<bound method Series.unique of 0          90
2         115
3         262
4         110
5         310
         ... 
160609     75
160610     75
160611    120
160614    150
160615     88
Name: param_moc, Length: 106494, dtype: int64>

In [0]:
## cecha param_rok-produkcji po przekszałceniu
## cecha param_moc po przekszałceniu
feats = ['param_rok-produkcji',
         'param_napęd__cat',
         'param_stan__cat',
         'param_skrzynia-biegów__cat',
         'param_faktura-vat__cat',
         'param_moc',
         'param_marka-pojazdu__cat',
         'feature_kamera-cofania__cat',
         'param_typ__cat',
         'param_pojemność-skokowa__cat',
         'seller_name__cat',
         'feature_wspomaganie-kierownicy__cat',
         'param_model-pojazdu__cat',
         'param_wersja__cat',
         'param_kod-silnika__cat',
         'feature_system-start-stop__cat',
         'feature_asystent-pasa-ruchu__cat',
         'feature_czujniki-parkowania-przednie__cat',
         'feature_łopatki-zmiany-biegów__cat',
         'feature_regulowane-zawieszenie__cat']

In [65]:
""" wynik (-9716.874397429374, 61.855325308737015)"""
run_model(model,feats)

[07:47:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:48:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:48:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9716.874397429374, 61.855325308737015)

In [70]:
##parametr param_pojemność-skokowa
df['param_pojemność-skokowa'].unique()

array(['898 cm3', '1 560 cm3', '3 000 cm3', ..., '5 992 cm3', '1 966 cm3',
       '142 280 cm3'], dtype=object)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None'  else int(str(x).split('cm')[0].replace(' ','')))

In [0]:
## cecha param_rok-produkcji po przekszałceniu
## cecha param_moc po przekszałceniu
## cecha param_pojemność-skokowa po przekszałceniu
feats = ['param_rok-produkcji',
         'param_napęd__cat',
         'param_stan__cat',
         'param_skrzynia-biegów__cat',
         'param_faktura-vat__cat',
         'param_moc',
         'param_marka-pojazdu__cat',
         'feature_kamera-cofania__cat',
         'param_typ__cat',
         'param_pojemność-skokowa',
         'seller_name__cat',
         'feature_wspomaganie-kierownicy__cat',
         'param_model-pojazdu__cat',
         'param_wersja__cat',
         'param_kod-silnika__cat',
         'feature_system-start-stop__cat',
         'feature_asystent-pasa-ruchu__cat',
         'feature_czujniki-parkowania-przednie__cat',
         'feature_łopatki-zmiany-biegów__cat',
         'feature_regulowane-zawieszenie__cat']

In [77]:
""" wynik (-9569.367265637287, 71.91237464960756)"""
run_model(model,feats)

[08:02:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:02:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:02:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.367265637287, 71.91237464960756)